In [66]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()


In [67]:
class WideAndDeep(tf.keras.Model):
    """
    非常简单的wide and deep模型
    """
    def __init__(self,name,emb_input_output_dims,deep_units_list,first_layer_units):
        """

        :param name: string 模型名称
        :param emb_input_output_dims: [(int,int)] 用于构建embedding_layer的参数，每个tuple包含一个input_dim和一个output_dim，基于此构建embedding lyaer
        :param deep_units_list: [int] 深度模型各个layer的神经元个数
        :param first_layer_units: int 最上层的dense层的神经元个数，即输出的维度
        """
        super(WideAndDeep,self).__init__(name=name)
        self.var=tf.Variable([1.1]) # 没啥用就是测试一下
        self.emb_layers=list()
        for input_dim,output_dim in emb_input_output_dims:
            self.emb_layers.append(tf.keras.layers.Embedding(input_dim=input_dim,output_dim=output_dim))

        self.deep_layers=list()
        for units in deep_units_list:
            self.deep_layers.append(
                tf.keras.layers.Dense(units=units,
                                              kernel_initializer=tf.keras.initializers.TruncatedNormal(),
                                              bias_initializer=tf.keras.initializers.Ones())
            )
        self.first_layer=tf.keras.layers.Dense(units=first_layer_units,
                                               activation=None,
                                              kernel_initializer=tf.keras.initializers.TruncatedNormal(),
                                              bias_initializer=tf.keras.initializers.Ones())

    def call(self, wide_input_batch,deep_input_batch,deep_emb_input_batch):
        """

        :param wide_input_batch: tensor or a batch of data 广度模型的输入数据batch 不会经过embedding，shape:[batch_size, wide_input_feature_dim]
        :param deep_input_batch: tensor or a batch of data 深度模型的不需要embedding的输入数据batch，shape:[batch_size, deep_input_feature_dim]
        :param deep_emb_input_batch: tensor or a batch of data 深度模型的不需要embedding的输入数据batch，shape:[batch_size, deep_emb_input_feature_dim]
        :return:
        """

        emb_outputs=list()
        for i,emb_layer in enumerate(self.emb_layers):
            emb_outputs.append(emb_layer(deep_emb_input_batch[:,i]))

        concat_emb_output=tf.concat(emb_outputs,axis=1)
        deep_input=tf.concat([concat_emb_output,deep_input_batch],axis=1)
        for deep_layer in self.deep_layers:
            deep_input=deep_layer(deep_input)

        first_layer_input=tf.concat([wide_input_batch,deep_input],axis=1)
        output=self.first_layer(first_layer_input)
        return output

In [73]:
import numpy as np
wide_input_ds=tf.data.Dataset.from_tensor_slices(np.random.random(size=(10,10)).astype(np.float32))
batched_wide_input_ds=wide_input_ds.batch(5)
wide_input_batch=next(iter(batched_wide_input_ds))

deep_input_ds=tf.data.Dataset.from_tensor_slices(np.random.random(size=(10,10)).astype(np.float32))
batched_deep_input_ds=deep_input_ds.batch(5)
deep_input_batch=next(iter(batched_deep_input_ds))

deep_emb_input_ds=tf.data.Dataset.from_tensor_slices(np.random.randint(0,3,size=(10,1)))
batched_deep_emb_input_ds=deep_emb_input_ds.batch(5)
deep_emb_input_batch=next(iter(batched_deep_emb_input_ds))

wide_deep=WideAndDeep("name",emb_input_output_dims=[(3,2)],deep_units_list=[3,2],first_layer_units=2)
wide_deep.trainable=False
output=wide_deep(wide_input_batch,deep_input_batch,deep_emb_input_batch)
print(wide_deep.trainable_variables)
print(output)

[]
tf.Tensor(
[[0.7183332 1.0713701]
 [0.7878482 1.0301176]
 [0.7891807 1.0242541]
 [0.7828526 1.1120307]
 [0.7718165 1.0623623]], shape=(5, 2), dtype=float32)


[<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([1.1], dtype=float32)>]